In [1]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [2]:
#importing the dataset
with open('finale.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
data

,inputs,tags
0,hello,greeting
1,hi there,greeting
2,greetings,greeting
3,nice to meet you,greeting
4,heyoo,greeting
...,...,...
101,Mtu anaingia vipi Kwa live system,livesystem
102,Hello can I get assistance in the opening of t...,livesystem
103,Hello can I get the technicians to go about ac...,livesystem
104,How do I get to the live System,livesystem


In [3]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [4]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

34
number of unique words :  240
output length:  14


In [5]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x) #Can learn long-term dependencies between steps of data
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x) #Dense just takes input from previous layers
model  = Model(i,x)
#compiling the model 
#it checks for format errors and defines the loss functions
#not necessary for predicting
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
4/4 [==============================] - 3s 18ms/step - loss: 2.6424 - accuracy: 0.0283
Epoch 2/200
4/4 [==============================] - 0s 21ms/step - loss: 2.6283 - accuracy: 0.1509
Epoch 3/200
4/4 [==============================] - 0s 19ms/step - loss: 2.6146 - accuracy: 0.1509
Epoch 4/200
4/4 [==============================] - 0s 19ms/step - loss: 2.5992 - accuracy: 0.1604
Epoch 5/200
4/4 [==============================] - 0s 20ms/step - loss: 2.5780 - accuracy: 0.1509
Epoch 6/200
4/4 [==============================] - 0s 17ms/step - loss: 2.5523 - accuracy: 0.1509
Epoch 7/200
4/4 [==============================] - 0s 17ms/step - loss: 2.5189 - accuracy: 0.1509
Epoch 8/200
4/4 [==============================] - 0s 20ms/step - loss: 2.4889 - accuracy: 0.1509
Epoch 9/200
4/4 [==============================] - 0s 19ms/step - loss: 2.4563 - accuracy: 0.1509
Epoch 10/200
4/4 [==============================] - 0s 19ms/step - loss: 2.4304 - accuracy: 0.1604
Epoch 11/200
4/4 [=

In [ ]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Maya : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You : Hello
1/1 [==============================] - 0s 493ms/step
Maya :  Hey there! How can I help you with the CPIMS system today?
You : I need to reset my password
1/1 [==============================] - 0s 28ms/step
Maya :  Use the forgot password functionality.
You : I need to undo a double entry
1/1 [==============================] - 0s 31ms/step
Maya :  If in registry just delete but if in placement forward the CPIMS ids. Working on a permanent solution about it.


In [ ]:
plt.figure(figsize = (10,10))
plt.scatter(data['inputs'],data['tags'])
plt.savefig('output.jpg')
plt.xlabel('Inputs', fontsize = 50)
plt.ylabel('tags', fontsize = 50)
plt.title('Plot of tags against inputs', fontsize = 50)